In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/library-services-dataset/Library_Services_20240831.csv


In [2]:
data = pd.read_csv('/kaggle/input/library-services-dataset/Library_Services_20240831.csv')
print(data.head())

               Branch                                            Webpage  \
0          Aspen Hill  http://montgomerycountymd.gov/Library/branches...   
1        Gaithersburg  http://montgomerycountymd.gov/Library/branches...   
2  Maggie Nightingale  http://montgomerycountymd.gov/Library/branches...   
3             Potomac  http://montgomerycountymd.gov/Library/branches...   
4           White Oak  http://montgomerycountymd.gov/Library/branches...   

                           Address           City    Zip  Square Feet  \
0             4407 Aspen Hill Road      Rockville  20853        16100   
1  18330 Montgomery Village Avenue   Gaithersburg  20879        62525   
2              19633 Fisher Avenue    Poolseville  20837         6250   
3            10101 Glenolden Drive        Potomac  20854        16000   
4       11701 New Hampshire Avenue  Silver Spring  20904        16205   

   Inventory  Population Size         Phone     TTY Phone  ...  metrobus Q1  \
0      78647             

'include only "object" type columns in the statistical summary.

In [3]:
print(data.describe(include="object"))

            Branch                                            Webpage  \
count           21                                                 21   
unique          21                                                 21   
top     Aspen Hill  http://montgomerycountymd.gov/Library/branches...   
freq             1                                                  1   

                     Address       City         Phone     TTY Phone Wi-Fi  \
count                     21         21            21             8    21   
unique                    21         13            21             8     1   
top     4407 Aspen Hill Road  Rockville  240-773-9410  301-871-2097     y   
freq                       1          3             1             1    21   

       Charging Go Kits Literacy After School  ... metrobus j4 metrobus l8  \
count        21      21                    21  ...          21          21   
unique        1       1                     1  ...           2           2   
top           

We can see that in the 21 rows, there are 21 unique branch, 21 unique address, 13 unique city, and one unique charging.Some branch are very common. the same number for freq and unique in Location

In [4]:
data['input'] = 'TEXT1: ' + data.Location + '; TEXT2: ' + data.Address + '; ANC1: ' + data.Branch
print(data.input.head())

0    TEXT1: 4407 Aspen Hill Rd\nRockville 20853\n(3...
1    TEXT1: 18330 Montgomery Village Ave\nGaithersb...
2    TEXT1: 19633 Fisher Avenue\nPoolseville 20837\...
3    TEXT1: 10101 Glenolden Drive\nPotomac 20854\n(...
4    TEXT1: 11701 New Hampshire Avenue\n(39.047701,...
Name: input, dtype: object


we use Transformers it's a Dataset object to store a set of data, We can create one:

In [5]:
from datasets import Dataset,DatasetDict

dataset = Dataset.from_pandas(data)
print(dataset)

Dataset({
    features: ['Branch', 'Webpage', 'Address', 'City', 'Zip', 'Square Feet', 'Inventory', 'Population Size', 'Phone', 'TTY Phone', 'Sunday Open', 'Sunday Close', 'Monday Open', 'Monday Close', 'Tuesday Open', 'Tuesday Close', 'Wednesday Open', 'Wednesday Close', 'Thursday Open', 'Thursday Close', 'Friday Open', 'Friday Close', 'Saturday Open', 'Saturday Close', 'Wi-Fi', 'Charging', 'Go Kits', 'Literacy After School', 'Collaboration', 'Computers', 'Discovery Room', '3D printers', 'makerspace', 'E-Reader', 'Laptops', 'amharic adult', 'amharic child', 'chinese adult', 'chinese child', 'farsi adult', 'farsi child', 'french adult', 'french child', 'korean adult', 'korean child', 'russian adult', 'russian child', 'spanish adult', 'spanish child', 'vietnamese adult', 'vietnamese child', 'Metro Station', 'Bike Share 2', 'Bike Share 1', 'Bus 4', 'Bus 5', 'Bus 6', 'Bus 7', 'Bus 9', 'Bus 12', 'Bus 14', 'Bus 15', 'Bus 16', 'Bus 17', 'Bus 19', 'Bus 20', 'Bus 23', 'Bus 26', 'Bus 29', 'Bus 

Tokenization: Split each text up into words (or actually, as we'll see, into tokens)
Numericalization: Convert each word (or token) into a number.
For each text in the columns that contain textual information, we will divide the text into words or "tokens". Here is an example of using nltk or spacy for tokenizer:

In [6]:
import nltk
from nltk.tokenize import word_tokenize

# Downloading the necessary resources from nltk
nltk.download('punkt')

# Suppose the "Webpage" column contains text
data['tokens'] = data['Webpage'].apply(lambda x: word_tokenize(x.lower()))


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Next, we will convert each token into a unique number. To do this, we can use a dictionary which associates each token with a unique digital identifier.

In [7]:
from sklearn.preprocessing import LabelEncoder

# Initializing the encoder
label_encoder = LabelEncoder()

# Encoding of tokens
data['tokens_encoded'] = data['tokens'].apply(lambda x: label_encoder.fit_transform(x))


the columns that need to be tokenized and digitized. Columns like "Branch", "Address", or "City" could contain textual data to tokenize. You can filter these columns before launching the tokenization and numericalization process.

In [8]:
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

# load data
df = {
    'Branch': ['Branch A', 'Branch B'],
    'Webpage': ['https://branchA.com', 'https://branchB.com'],
    'Address': ['123 Main St', '456 Oak St'],
    'City': ['New York', 'Los Angeles']
}
data = pd.DataFrame(df)

# Tokenization of text columns
text_columns = ['Branch', 'Webpage', 'Address', 'City']
for col in text_columns:
    data[f'{col}_tokens'] = data[col].apply(lambda x: word_tokenize(x.lower()))

# Numericalization
label_encoder = LabelEncoder()
for col in text_columns:
    data[f'{col}_tokens_encoded'] = data[f'{col}_tokens'].apply(lambda x: label_encoder.fit_transform(x))

# Display result
print(df)


{'Branch': ['Branch A', 'Branch B'], 'Webpage': ['https://branchA.com', 'https://branchB.com'], 'Address': ['123 Main St', '456 Oak St'], 'City': ['New York', 'Los Angeles']}
